In [109]:
import spacy, random
from spacy.util import minibatch, compounding
from pathlib import Path
from spacy.training import Example


In [110]:
nlp = spacy.load('en_core_web_sm')

In [111]:
# TODO: After first tests promissing then add all of the other train_data

train_data = [
    ("Adjusted Mean Change From Baseline in Fasting Plasma Glucose at Week 24 (Last Observation Carried Forward [LOCF]", 
    {"entities":[(0,20, "Change"),(26,34,"Reference"),(38,61,"Variable"), (65,72,"Timepoint")]}),
    ("Number of Participants With Abnormal Electrocardiogram (ECG) Interval", 
    {"entities":[(0,22, "Change"),(28,69,"Variable")]}),
]

In [112]:
ner = nlp.get_pipe("ner")

In [113]:
# Add the new entities in to the pipeline

for _, annotations in train_data:
    for ent in annotations.get("entities"):
        ner.add_label(ent[2])

In [114]:
disable_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]

In [115]:
with nlp.disable_pipes(*disable_pipes):
    optimizer = nlp.resume_training()

    for iter in range(300):
        random.shuffle(train_data)
        losses = {}

        # TODO: Rewrite to fit batches
        batches = minibatch(train_data, size=compounding(1.0, 4.0, 1.001))
        
        for sentance in train_data:
            text = sentance[0]
            example = Example.from_dict(nlp.make_doc(text), sentance[1])
            nlp.update(
                [example],
                drop = 0.5,
                losses = losses,
                sgd = optimizer
            )
            print("Losses: ", losses)

    

Losses:  {'ner': 4.8937905059073294}
Losses:  {'ner': 10.578891751687278}
Losses:  {'ner': 4.177790986171999}
Losses:  {'ner': 7.568495790647287}


D:\dev_apps\conda\lib\site-packages\spacy\training\iob_utils.py:139: UserWarning: [W030] Some entities could not be aligned in the text "Adjusted Mean Change From Baseline in Fasting Plas..." with entities "[(0, 20, 'Change'), (26, 34, 'Reference'), (38, 61...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(


Losses:  {'ner': 4.507014386502671}
Losses:  {'ner': 9.241982218050303}
Losses:  {'ner': 4.973228678914893}
Losses:  {'ner': 8.894033113390524}
Losses:  {'ner': 4.779553721115235}
Losses:  {'ner': 9.537394045466856}
Losses:  {'ner': 6.151100738015787}
Losses:  {'ner': 9.278745710044477}
Losses:  {'ner': 3.256602846307068}
Losses:  {'ner': 6.904693904633465}
Losses:  {'ner': 3.4646663623957794}
Losses:  {'ner': 8.75994878453181}
Losses:  {'ner': 3.6604596863357606}
Losses:  {'ner': 7.242260483889785}
Losses:  {'ner': 3.1798384177185053}
Losses:  {'ner': 6.894306852481151}
Losses:  {'ner': 2.588118025594226}
Losses:  {'ner': 5.328763015719701}
Losses:  {'ner': 3.200478361096482}
Losses:  {'ner': 6.076461372554486}
Losses:  {'ner': 3.8033931088510826}
Losses:  {'ner': 7.5252315461163874}
Losses:  {'ner': 3.290253778691921}
Losses:  {'ner': 5.590635821630032}
Losses:  {'ner': 2.812244674203612}
Losses:  {'ner': 5.819183568846142}
Losses:  {'ner': 3.1155103588553175}
Losses:  {'ner': 8.1526

In [117]:
for text, _ in train_data:
        target = nlp(text)
        for entity in target.ents:
                print(entity.text, entity.start_char, entity.end_char, entity.label_)
                # displacy.render(nlp(target.text), style='ent')

Adjusted Mean Change 0 20 Change
Baseline 26 34 Reference
Glucose at Week 53 68 Change
Number of Participants 0 22 Change
Abnormal Electrocardiogram (ECG) 28 60 Variable


In [118]:
from spacy import displacy

target = nlp("Number of Participants with undiagnosed type 2 diabetes")
for entity in target.ents:
    print(entity.text, entity.start_char, entity.end_char, entity.label_)
displacy.render(nlp(target.text), style='ent')

Number of Participants 0 22 Change
